### JDBC Drivers
1. Type 1, JDBC-ODBC bridge driver
2. Type 2, Native API driver
3. Type 3, network protocol driver
4. Type 4, pure Java driver (thin driver)

### Connection Strings
- **Derby:** jdbc:derby:\[subsubprotocol:\]\[databaseName\]\[;attribute=value\]*
- **MySQL:** jdbc:mysql://\[host\]\[,failoverhost...\]\[:port\]/\[database\]\[?property1=value1\]\[&property2=value2\]
- **Oracle:** jdbc:oracle:thin:\[@host\]\[:port\]\[:sid\] or TNS syntax

### How a Connection is Made?
Most of the jdbc identifiers like `Statement`, `PreparedStatement`, `ResultSet`, etc are interfaces. The driver provides the actual implementation. For example:
- Connection(I) -- T4CConnection (C)
- Statement(I) -- OracleStatement(I) -- T4CStatement(C)

Below is snippet of how a connection is obtained
```java
public static void main(String args[]){
    String url = someUrl;
    String username = "myuser";
    String password = "mypassword";
    
    try{
        Class.forName("oracle.jdbc.driver.OracleDriver");        
        Connection conn = DriverManager.getConnection(url, username, password);
    } catch(SQLException e){
        e.printStackTrace();
    } catch(ClassNotFoundException e){
        e.printStackTrace();
    }
}
```

The first statement `Class.forName` searches for the class in the CLASSPATH and if found, executes the static initializer block in that class. The static initializer code registers the driver with `DriverManager` using `DriverManager.registerDriver`. Now whenever a call to `DriverManager.getConnection` is made, the DriverManager searches among registered driver and returns appropriate connection object.  
Since JDBC 4.0, Class.forName is not required, DriverManager itself does that bit.

### DataSource and Connection Pool
Before we discuss DataSource, we should be aware of JNDI (Java Naming and Directory Interface). And before we discuss JNDI, we need to know about naming services. A naming service is a mapping between a name and object (or reference to the object). For example, DNS (Domain Name Server) is a naming service which maps a URL to the IP address of the host. Similarly, LDAP is another naming server. The problem is that each naming service has its own naming convention. For example, DNS has names like `www.google.com`, whereas LDAP has name like `cn=Steve Jobs, o=Contose, c=US`. JNDI utilises a consistent naming convention and provides a common interface to existing naming services.  

**JNDI** Central to JNDI are:
- `Name` class which represents an ordered sequence of subnames
- `Context` interface which represents a set of bindings

Concrete implementation of Context interface must provide implementations for binding, unbinding, renaming, listing all bindings, etc. A context may also have subcontexts. `IntialContext` is an implementation of Context interface and  serves as our entry point to a naming system. 

**InitialContext** The InitialContext constructor takes a set of properties, in the form of a Hashtable or one of its subclasses, such as a Properties object.
```java
Properties props = new Properties(); 
props.put(Context.INITIAL_CONTEXT_FACTORY, "com.sun.jndi.fscontext.RefFSContextFactory"); 
props.put(Context.PROVIDER_URL, "file:///"); 
Context initialContext = new InitialContext(props);
```

`Context.INITIAL_CONTEXT_FACTORY` property states the factory class in a JNDI service provider. In simpler terms it specifies the naming service we are trying to use. The rest of the properties are naming server specific and may include things such as password.  

Once the Context is ready, we can use it to lookup the name object mapping:
```java
Object obj = initialContext.lookup(name);
```
**DataSource** ConnectionManager is a lower level API, DataSource on the other hand is higher level and the main advantage of using DataSource is that it provides *Connection Pooling* and *Distributed Transactions*. On bigger projects DataSource should be the preferred way of connecting to the database. The DataSource interface is implemented by a driver vendor and the driver vendor can:
- create a basic implementation with no connection pooling or distributed transaction
- create implementation supporting the above listed features

Another big advantage of using DataSource is that you can use it in conjuntion with JNDI, therefore database connection details are hidden away. A datasource object can be created and mapped in JNDI or you can do that in your server's admin console:
```java
// Create datasource
BasicDataSource ds = new BasicDataSource();
ds.setServerName("development");
ds.setDatabaseName("CUSTOMERS");
ds.setDescription("Customer database");

// Registering it to naming service
Context ctx = new InitialContext();
ctx.bind("jdbc/customerDb", ds);
```

Now whenever we want to get connections
```java
Context ctx = new InitialContext();
DataSource ds = (DataSource) ctx.lookup("jdbc/customerDb");
Connection con = ds.getConnection();
```

### DataBase Metadata
To get information about the database, use the `DatabaseMetaData` object.
```java
DatabaseMetaData metadata = conn.getMetaData();
System.out.println("URL: " + metadata.getURL());
System.out.println("Product: " + metadata.getDatabaseProductName());
System.out.println("Version: " + metadata.getDatabaseProductVersion());
System.out.println("Driver Name: " + metadata.getDriverName());
System.out.println("Driver Version: " + metadata.getDriverVersion());
System.out.println("Max Connections: " + metadata.getMaxConnections());
```

### Statements and ResultSet
**Statement** object creates query and executes it.
```
Statement
        |
        +-- PreparedStatement: prevents SQL injection, is pre-compiled
        +-- CallableStatement: used to run stored procedures
```

**ResultSet** object is a pointer to the results in the database. It loads records in blocks from database. Set the block size using `setBlockSize` method.  

ResultSet types:
1. TYPE_FORWARD_ONLY: can move forward only. Calling previous methods will lead to SQLException. DB changes made after query is not reflected.
2. TYPE_SCROLL_INSENSITIVE: can move forward/backward or any absolute position. DB changes made after query is not reflected.
3. TYPE_SCROLL_SENSITIVE: same as above except that it reflects db changes.

ResultSet cncurrency:
1. CONCUR_READ_ONLY: only read data
2. CONCUR_UPDATABLE: can update ResultSet

Some sample code:  
**Statment demo:**
```java
public void statementDemo(){
    try(Statement st = conn.createStatement()){
        String sql = "SELECT * FROM CUSTOMERS";
        boolean type = st.execute(sql);    // if type is true, this means we get a ResultSet
                                           // if it is false, we get update count
        if(type){
            try(ResultSet rs = st.getResultSet()){
                while(rs.next()){
                    System.out.println("Name: " + rs.getString(1));
                    System.out.println("Age: " + rs.getInt(2));
                }
            }
        }
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

**PreparedStatment demo:**
```java
public void preparedStatementDemo(){
    String sql = "SELECT * FROM CUSTOMERS WHERE NAME = ?";
    try(PreparedStatement ps = conn.prepareStatement(sql)){
        ps.setString(1, "John Doe");
        
        try(ResultSet rs = ps.executeQuery()){
            while(rs.next()){
                System.out.println("Name: " + rs.getString(1));
                System.out.println("Age: " + rs.getInt(2));
            }
        }
        
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

**CallableStatment demo:**
```java
public void callableStatementDemo(){
    String sql = "CALL PKG_GET_ALL_ALERTS(?,?)";
    try(CallableStatement cs = conn.prepareCall(sql)){
        cs.setString(1, 12653);
        cs.registerOutParameter(9, OracleTypes.CURSOR);
        cs.execute();
        
        try(ResultSet rs = (ResultSet) cs.getObject(2)){
            while(rs.next()){
                System.out.println("Alert Id: " + rs.getString(1));
                System.out.println("Alert Count: " + rs.getInt(2));
            }
        }
        
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

**Updating Rows on the Fly:**
```java
public void updatableResultSet(){
    // First check if the database supports these operations
    try{
        DatabaseMetaData metadata = conn.getMetaData();
        if(metadata.supportsResultSetType(ResultSet.TYPE_SCROLL_INSENSITIVE))
            System.out.println("Supports TYPE_SCROLL_INSENSITIVE");
        if(metadata.supportsResultSetConcurrency(ResultSet.TYPE_SCROLL_INSENSITIVE,
                                                 ResultSet.CONCUR_UPDATABLE))
            System.out.println("Supports CONCUR_UPDATABLE");
    }
    
    String sql = "SELECT * FROM CUSTOMERS WHERE NAME = ?";
    try(PreparedStatement ps = conn.prepareStatement(sql)){
        ps.setString(1, "John Doe");
        
        try(ResultSet rs = ps.executeQuery()){
            while(rs.next()){
                rs.updateString("NAME", "Mr. " + rs.getString(1));
                rs.updateRow();
                // can also delete this row using rs.deleteRow()
            }
        }
        
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

**Batch Update:** 
```java
public void batchDemo(){
    String sql = "INSER INTO USER(NAME, AGE) VALUES(?,?)";
    try(PreparedStatement ps = conn.prepareStatement(sql)){
        ps.setString(1, "John Doe");
        ps.setInt(2,26);
        ps.addBatch();
        
        ps.setString(2, "Jane Doe");
        ps.setInt(2,22);
        ps.addBatch();
        
        int counts[] = ps.executeBatch(); // counts {1,1}
        
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

### ResultSetMetaData
Is useful when we want to get column names of the cursor.
```java
public void printHeaders(PreparedStatemet ps){
    try(ResultSet rs = ps.executeQuery()){
        ResultSetMetaData metadata = rs.getMetaData();
        int numberOfColumns = metadata.getColumnCount();
        for(int i=0; i<numberOfColumns; i++){
            System.out.print(metadata.getColumnName(i) + " ");
        }
    } catch(SQLException e){
        e.printStackTrace();
    }
}
```

### Transactions
By default database connections are in auto-commit mode. This means that commit is automaticaly done after every SQL execution. This shouldn't be the case specially when batch is involved (so as to be able to rollback).

When auto-commit is false, all SQL statements before call to commit are included in a transaction and commited as a unit when commit is called. In transaction rows being accessed are locked and cannot be accessed by others.

```java
public void transactionDemo(){
    String insert = "INSERT INTO FOOD(ITEM, PRICE) VALUES('Coffee', 25.55)";
    String update = "UPDATE FOOD SET PRICE = PRICE + 2.10";
    try{
        conn.setAutoCommit(false);
    } catch (SQLException e){
        e.printStackTrace();
    }
    
    try(Statement is = conn.createStatement();
        Statment us = conn.createStatement();){
        is.executeUpdate(insert);
        us.executeUpdate(update);
        conn.commit();
    } catch(SQLException e){
        try{
            conn.rollback()
        } catch (SQLException e){
            e.printStackTrace();
        }
    }
}
```